In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
test_id = test.id
train.drop(['id'], axis=1, inplace=True)
test.drop(['id'] ,axis=1, inplace=True)

X_train = train.drop(['target'], axis=1)
y_train = train.target
X_test = test

feature engineering

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [6]:
h_skew = X_train.loc[:, train.skew() >= 2].columns # with skewed
l_skew = X_train.loc[:, train.skew() < 2].columns # Bimodal

# high skewd columns:
# add mean, median, skew, min, max, var, std
X_train['median_h'] = X_train[h_skew].median(axis=1)
X_test['median_h'] = X_test[h_skew].median(axis=1)
X_train['var_h'] = X_train[h_skew].var(axis=1)
X_test['var_h'] = X_test[h_skew].var(axis=1)

# bimodal columns:

X_train['mean_l'] = X_train[l_skew].mean(axis=1)
X_test['mean_l'] = X_test[l_skew].mean(axis=1)
X_train['std_l'] = X_train[l_skew].std(axis=1)
X_test['std_l'] = X_test[l_skew].std(axis=1)
X_train['median_l'] = X_train[l_skew].median(axis=1)
X_test['median_l'] = X_test[l_skew].median(axis=1)
X_train['skew_l'] = X_train[l_skew].skew(axis=1)
X_test['skew_l'] = X_test[l_skew].skew(axis=1)
X_train['max_l'] = X_train[l_skew].max(axis=1)
X_test['max_l'] = X_test[l_skew].max(axis=1)
X_train['var_l'] = X_train[l_skew].var(axis=1)
X_test['var_l'] = X_test[l_skew].var(axis=1)

model

In [7]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks
import tensorflow as tf
import random
import os
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, RobustScaler, PowerTransformer

# seed
seed = 42
def seedAll(seed):
    np.random.seed(seed) # set numpy seed
    tf.random.set_seed(seed) # set tensorflow seed
    random.seed(seed) # set random seed
    os.environ['PYTHONHASHSEED'] = str(seed)
seedAll(seed)

In [18]:
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=20,
    min_delta=0,
    baseline=None,
    restore_best_weights=True,
    verbose=0
)
plateau = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    mode='min',
    patience=7,
    factor=.2,
    verbose=0        
)

def create_model(each_layers, activation):   
    model = keras.Sequential()
    for layer in range(len(each_layers)):
        if layer == 0:
            model.add(layers.Dense(each_layers[layer], activation=activation, input_shape=[X_train.shape[1]]))
        else:
            model.add(layers.Dense(each_layers[layer], activation=activation))
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.0007),
        loss='binary_crossentropy',
        metrics=['AUC']
    )
    return model

In [19]:
model_nn = KerasClassifier(build_fn=create_model)

<ipython-input-19-1ba7203fdc6e>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model_nn = KerasClassifier(build_fn=create_model)


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_valid = train_test_split(X_train, test_size=0.2)

In [17]:
y_train, y_valid = y_train[X_train.index], y_train[X_valid.index]

In [34]:
params_fit_nn = {
    'epochs': [100],
    'batch_size': [2048],
    'verbose': [1],
    'each_layers': [[16, 4]],
    'activation': ['swish'],
    'validation_data': [(X_valid, y_valid)]
}
params_nn = {
    'each_layers': [[16, 4], [32, 16], [64, 32]],
    'activation': ['swish']
}

search_nn = GridSearchCV(model_nn,
                         params_fit_nn,
                         cv=2,
                         n_jobs=-1,
                         verbose=1
                        ).fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [nan]
  warnings.warn(


Epoch 1/100
235/235 [==============================] - 1s 3ms/step - loss: 0.8323 - auc: 0.5026 - val_loss: 0.6934 - val_auc: 0.5056
Epoch 2/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6931 - auc: 0.5075 - val_loss: 0.6927 - val_auc: 0.5131
Epoch 3/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6920 - auc: 0.5220 - val_loss: 0.6905 - val_auc: 0.5378
Epoch 4/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6878 - auc: 0.5652 - val_loss: 0.6857 - val_auc: 0.5803
Epoch 5/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6782 - auc: 0.6078 - val_loss: 0.6710 - val_auc: 0.6175
Epoch 6/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6635 - auc: 0.6255 - val_loss: 0.6764 - val_auc: 0.5871
Epoch 7/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6669 - auc: 0.6203 - val_loss: 0.6587 - val_auc: 0.6458
Epoch 8/100
235/235 [==============================] - 1s 2ms/step - 

235/235 [==============================] - 1s 2ms/step - loss: 0.6063 - auc: 0.7410 - val_loss: 0.6055 - val_auc: 0.7456
Epoch 63/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6041 - auc: 0.7427 - val_loss: 0.6040 - val_auc: 0.7437
Epoch 64/100
235/235 [==============================] - 1s 3ms/step - loss: 0.6050 - auc: 0.7416 - val_loss: 0.6056 - val_auc: 0.7428
Epoch 65/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6057 - auc: 0.7409 - val_loss: 0.6091 - val_auc: 0.7399
Epoch 66/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6043 - auc: 0.7415 - val_loss: 0.6020 - val_auc: 0.7436
Epoch 67/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6055 - auc: 0.7406 - val_loss: 0.6008 - val_auc: 0.7457
Epoch 68/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6029 - auc: 0.7426 - val_loss: 0.6018 - val_auc: 0.7456
Epoch 69/100
235/235 [==============================] - 1s 2ms/step - loss: